In [635]:
import numpy as np
import random
import cv2
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

import import_ipynb
from Server_code import *

In [480]:
#split data into training and test set
df = pd.read_csv('C:\\Users\\VasilisFermelis\\Desktop\\FL.csv')

X = df.iloc[:,1:12]
y = df.iloc[:,0] 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

image_list = df.iloc[:,1:12]
label_list = df.iloc[:,0] 

In [481]:
#create clients
clients = create_clients(X_train, y_train, num_clients=10, initial='client')

In [483]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, df) in clients.items():
    clients_batched[client_name] = batch_data(df)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

comms_round = 100
batch_data(df)

<BatchDataset element_spec=TensorSpec(shape=(None, 10), dtype=tf.string, name=None)>

In [497]:
comms_round = 100
#create optimizer
lr = 0.1
loss='mse'
metrics = ['accuracy']
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               ) 

C:\Users\VasilisFermelis\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [514]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(9, 1)

#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(9, 1)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

In [634]:
#fit local model with client's data
local_model.fit(X_train, y_train, batch_size=64, epochs=20, verbose=0)

In [521]:
#scale the model weights and add to list
scaling_factor = weight_scalling_factor(clients_batched, client)
scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
scaled_local_weight_list.append(scaled_weights)
        
#clear session to free memory after each communication round
K.clear_session()
          
#to get the average over all the local model, we simply take the sum of the scaled weights
average_weights = sum_scaled_weights(scaled_local_weight_list)
    
#update global model 
global_model.set_weights(average_weights)

In [529]:
local_model.evaluate(X_test, y_test)
local_model.predict(X_test)

94/94 [==============================] - 0s 1ms/step


array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [578]:
new_sample = [[55,18,1,2022,8,9,14,15,38]]
output = local_model.predict(new_sample)
print(output)

1/1 [==============================] - 0s 26ms/step
[[0.]]
94/94 [==============================] - 0s 1ms/step


array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [633]:
#test global model and print out metrics after each communications round
for(X_test, Y_test) in test_batched:
    global_loss = test_model(X_test, y_test, global_model, comm_round)

94/94 [==============================] - 0s 1ms/step
global_loss: 64.64874
